## FIT3182: Assignment 2 Part B (Event Producer 3) 

### Name: Ashley Ooi Yan-Lin (ID: 31171095)

### Task 1: Processing Data Stream

### (c) Write a python program that loads all the data from hotspot_TERRA_streaming.csv and randomly (with replacement) feeds the data to the stream every 2 seconds. TERRA is another satellite from NASA that reports latitude, longitude, confidence and surface temperature of a location. You will need to append additional information such as producer information to identify the producer and created date & time.

Firstly, we need to establish a connection with our MongoClient and retrieve the collection created in Part A to obtain the most recent date.

In [1]:
import pymongo
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
from datetime import datetime,timedelta

client = MongoClient () 
db = client.fit3182_assignment_db
collection = db.partA

Next, we need to create our program to facilitate the transmission of our data to Kafka.

In [ ]:
from time import sleep
from json import dumps
from kafka import KafkaProducer
import random

# Reads data from hotspot_TERRA_streaming.csv, puts each row data into a document and appends all such documents into a list.
def read_hotspot_TERRA_streaming():
    hotspot_TERRA_streaming = pd.read_csv('hotspot_TERRA_streaming.csv')

    data = []
    for index,terraRow in hotspot_TERRA_streaming.iterrows():
        document = {}
        document['latitude'] = float(terraRow['latitude'])
        document['longitude'] = float(terraRow['longitude'])
        document['confidence'] = int(terraRow['confidence'])
        document['surface_temperature_celcius'] = int(terraRow['surface_temperature_celcius'])
        data.append(document)
    
    return data

# Gets the latest date in our collection
def get_latest_date():
    latest_date = collection.aggregate([
                {"$sort":{"date":-1}},
                {"$project":{"_id":0,"date":1}},
                {"$limit":1}
                ])
    for document in latest_date:
        latest_date = document['date']
    return latest_date
    
# Publishes message to Kafka
def publish_message(producer_instance, topic_name, data):
    try:
        producer_instance.send(topic_name, value=data)
        print('Message published successfully. Data: ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  value_serializer=lambda x: dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    
if __name__ == '__main__':
   
    topic = 'PartB'
    print('Publishing records..')
    producer03 = connect_kafka_producer()
    data = read_hotspot_TERRA_streaming() # Gets all the documents produced from hotspot_TERRA_streaming.csv
    latest_date = get_latest_date() + timedelta(days=1) # After getting latest date, we would add one day to it to get the first date we would use to start feeding data
    secondsPassed= 0 # Tracks how many seconds we should add to our latest date

    while True:
        chosenData = random.choice(data)   # Randomly chooses a document from our list of documents
        curr_date = latest_date + timedelta(seconds=secondsPassed) # Creates the date we will use to feed our data by adding the number of seconds to our latest date from Part A
        chosenData['producer'] = "hotspot_TERRA_streaming"
        chosenData["created_datetime"] = curr_date.strftime("%d/%m/%Y %H:%M:%S")
        publish_message(producer03, topic, chosenData)
        secondsPassed += 17280 # After we insert a hotspot streaming data, add 17280 seconds to secondPassed. 17280 second is equivalent to 24 hours divided by 5. As we would insert 5 hotspot AQUA streaming data per day, adding 17280 seconds each time after we insert a hotspot streaming data simulates the time difference of adding the 5 data in a day
        sleep(2) # Sleep for 2 seconds so that we would be able to insert 5 data per climate streaming data being inserted, as climate streaming data is inserted every 10 seconds

Message published successfully. Data: {'latitude': -37.8276, 'longitude': 142.7573, 'confidence': 73.0, 'surface_temperature_celcius': 47.0, 'created_datetime': '2022-05-20 03:23:17', 'producer_id': 'hotspot_terra_producer'}
Message published successfully. Data: {'latitude': -37.7134, 'longitude': 143.3818, 'confidence': 85.0, 'surface_temperature_celcius': 59.0, 'created_datetime': '2022-05-20 06:56:44', 'producer_id': 'hotspot_terra_producer'}
Message published successfully. Data: {'latitude': -36.9294, 'longitude': 142.7087, 'confidence': 78.0, 'surface_temperature_celcius': 51.0, 'created_datetime': '2022-05-20 08:17:22', 'producer_id': 'hotspot_terra_producer'}
Message published successfully. Data: {'latitude': -38.1756, 'longitude': 143.9829, 'confidence': 65.0, 'surface_temperature_celcius': 53.0, 'created_datetime': '2022-05-20 15:34:08', 'producer_id': 'hotspot_terra_producer'}
